In [1]:
import pickle
from glob import glob

files = sorted(glob('/home/husein/ssd2/processed-youtube-asr-whisper-large-test-set/*.pkl'))
len(files)

26

In [2]:
from datasets import Audio
from tqdm import tqdm
import numpy as np
import IPython.display as ipd

In [3]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [4]:
with open(files[0], 'rb') as fopen:
    data = pickle.load(fopen)

In [5]:
max(data[0]['asr_model'][0], key=data[0]['asr_model'][0].get)

'ms'

In [6]:
data[0]

{'wav_data': '/home/husein/ssd2/processed-youtube-asr-whisper-large-test-set/Ajaran_Gila_Kitab_Talmud_[paO9Jf-paPw]/0.mp3',
 'start': 0.0,
 'asr_model': ({'kk': 1.390142733725952e-05,
   'haw': 0.0004717402916867286,
   'ta': 4.902168075204827e-05,
   'br': 4.8484722356079146e-05,
   'sn': 0.0001415890292264521,
   'et': 4.332884600444231e-06,
   'sr': 4.682869985117577e-06,
   'gl': 1.590410761309613e-06,
   'cs': 8.45248214318417e-05,
   'nn': 0.007512533105909824,
   'es': 2.7303754905005917e-05,
   'fi': 2.3270855308510363e-05,
   'tr': 0.0009716476779431105,
   'bn': 5.781698928331025e-05,
   'ba': 3.542486304297654e-10,
   'fa': 0.00021259349887259305,
   'ar': 0.0002820058725774288,
   'la': 0.010719397105276585,
   'bs': 4.124826318729902e-06,
   'ja': 0.00048791812150739133,
   'uk': 0.0001976174971787259,
   'hu': 2.392162787145935e-05,
   'el': 6.403267616406083e-05,
   'pa': 7.194006320787594e-05,
   'mr': 3.670094156404957e-05,
   'ps': 8.641230124339927e-06,
   'kn': 2.85

In [7]:
ms_score = [s['avg_logprob'] for s in data[0]['asr_model'][1]['segments']]
en_score = [s['avg_logprob'] for s in data[0]['asr_model'][2]['segments']]

if np.mean(ms_score) > np.mean(en_score):
    selected = data[0]['asr_model'][1]['segments']
else:
    selected = data[0]['asr_model'][2]['segments']

In [8]:
i = 0

s = selected[i]['start']
e = selected[i]['end']
t = selected[i]['text']
s, e, t

(0.0,
 4.86,
 ' South Africa pernah mereka berikan Talmud, satu agama barat Kristian.')

In [9]:
f = data[0]['wav_data']
audio = reader.decode_example(reader.encode_example(f))['array']
y = audio[int(s * sr): int(e * sr)]

In [10]:
ipd.Audio(y, rate = sr)

In [11]:
import malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [12]:
vad = malaya_speech.vad.webrtc()

In [13]:
y_int = malaya_speech.astype.float_to_int(y)
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr))
np.mean([vad(frame) for frame in frames_int])

0.9876543209876543

In [14]:
import malaya
from malaya.text.normalization import cardinal

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)

In [15]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    tokenized = tokenizer.tokenize(string)
    string = ' '.join(tokenized)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [16]:
directory = '/home/husein/ssd3/stt/whisper-test-set'

In [17]:
# !rm -rf {directory}
!mkdir {directory}

mkdir: cannot create directory ‘/home/husein/ssd3/stt/whisper-test-set’: File exists


In [18]:
import torchaudio
import torch
import os

In [19]:
dataset = []
index = 0
for no, f in enumerate(files):
    
    with open(f, 'rb') as fopen:
        data = pickle.load(fopen)
    
    for i in tqdm(range(len(data))):
        lang = max(data[i]['asr_model'][0], key=data[i]['asr_model'][0].get)
        if lang not in {'ms', 'en'}:
            continue
        
        ms_score = [s['avg_logprob'] for s in data[i]['asr_model'][1]['segments']]
        en_score = [s['avg_logprob'] for s in data[i]['asr_model'][2]['segments']]

        if np.mean(ms_score) > np.mean(en_score):
            selected = data[i]['asr_model'][1]['segments']
        else:
            selected = data[i]['asr_model'][2]['segments']
            
        audio = reader.decode_example(reader.encode_example(data[i]['wav_data']))['array']
        for k in range(len(selected)):
            s = selected[k]['start']
            e = selected[k]['end'] + 0.1
            t = selected[k]['text']
            cleaned_t = preprocessing_text(t)
            if not len(cleaned_t):
                continue
                
            audio_path = os.path.join(directory, f'{index}.mp3')
            a = audio[int(s * sr): int(e * sr)]
            torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
            try:
                y_int = malaya_speech.astype.float_to_int(a)
                frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr, append_ending_trail = False))

                dataset.append(
                    {
                        'file': audio_path,
                        'text': t,
                        'cleaned': cleaned_t,
                        'no': no,
                        'i': i,
                        'k': k,
                        'avg_logprob': selected[k]['avg_logprob'],
                        'no_speech_prob': selected[k]['no_speech_prob'],
                        'vad': float(np.mean([vad(frame) for frame in frames_int])),
                    }
                )
            except:
                pass
            index += 1

100%|█████████████████████████████████████████| 190/190 [00:39<00:00,  4.79it/s]


In [20]:
len(dataset)

25664

In [21]:
import json

with open('dataset-whisper-stt-test-set.json', 'w') as fopen:
    json.dump(dataset, fopen)

In [22]:
!ls -lh dataset-whisper-stt-test-set.json

-rw-r--r-- 1 husein husein 7.2M Jul  17 17:07 dataset-whisper-stt-test-set.json


In [23]:
mp3s = glob('/home/husein/ssd3/stt/whisper-test-set/*.mp3')
len(mp3s)

25688

In [24]:
lens = []
for f in tqdm(mp3s):
    audio = reader.decode_example(reader.encode_example(f))['array']
    lens.append(len(audio) / sr)

100%|████████████████████████████████████| 25688/25688 [00:36<00:00, 699.69it/s]


In [25]:
sum(lens) / 60 / 60

19.558580000001424